In [1]:
import re
import pdfplumber
import json
import pandas as pd

**Specify The PDF File**

In [2]:
PDF_FILE = '2 DR BREAKFAST W- KIDS.pdf'

**Create JSON FILE**

In [3]:
def create_json_map():
    
    with pdfplumber.open(PDF_FILE) as pdf:
        pages = pdf.pages
        shelf_record = []
        page_shelf_record_dict = {}
        shelf_re = 'Shelf|CC Rack|Deleted Items- Adjust|Deleted Item|Added Item'
        for i,page in enumerate(pages):

            print("[INFO] Processed",page)    
            if i>0:
                page_number = "page_"+str(i+1)
                text = page.extract_text()
                shelf_record_dict = {}
                for line in text.split('\n'):
                    shelf_number = re.findall(shelf_re, line, flags=re.IGNORECASE)

                    if shelf_number:
                        shelf_num_index = re.search(shelf_number[0]+'\s*([a-zA-Z0-9_.-]*)',line).group()
                        shelf_num_width =  re.search('Width:'+'\s*([a-zA-Z0-9_.-.\']*)',line).group()
                        shelf_num_depth = re.search('Depth:'+'\s*([a-zA-Z0-9_.-.\']*)\s*([a-zA-Z0-9_.-.\']*)',line).group()
                        shelf_record_dict[shelf_num_index] = {"Width:":shelf_num_width.split()[1], "Depth":shelf_num_depth.split(': ')[1]}

                        shelf_record.append(shelf_num_index)
                page_shelf_record_dict[page_number] = shelf_record_dict 
#         print(page_shelf_record_dict)
        
        dict_to_json = json.dumps(page_shelf_record_dict)
        print("Successfully created JSON File!!")

        json_file = PDF_FILE.split('.')[0] + ".shelves.json"
        with open(json_file, "w") as f:
            f.write(dict_to_json)
        #     print(shelf_record)

**Extract Table Information**

In [4]:
def get_table_info(index,l):
    l= l.rstrip()
    l= l.lstrip()

    loc_id = l.split()[0]
    l = ' '.join(l.split(' ')[1:])
    check_int = re.findall('\d+',loc_id)
    if not check_int:
        return []

    facing = l.split()[-1]
    l = ' '.join(l.split(' ')[:-1])

    stockcode = l.split()[-1]
    l = ' '.join(l.split(' ')[:-1])

    bar_code = l.split()[-1]
    l = ' '.join(l.split(' ')[:-1])
    size_uom = l.split()[-1]  #only split last one and then extract size and uom
    l = ' '.join(l.split(size_uom)[:-1])
    size =''
    uom = ''
    
    if re.search('\d+\.\d+',size_uom) :
        size = re.search('\d+\.\d+',size_uom).group()
    else:
        return []
#         print("pattern not matched!!!")
    if size:
        uom =size_uom.split(size)[-1]
    else:
        return []
    name = l
    line_desc = [index,loc_id, name, size, uom, bar_code, stockcode, facing]
    return line_desc


**Get The JSON List and JSON File**

In [5]:
create_json_map()

[INFO] Processed <Page:1>
[INFO] Processed <Page:2>
[INFO] Processed <Page:3>
[INFO] Processed <Page:4>
[INFO] Processed <Page:5>
Successfully created JSON File!!


**Collecing Raw Data**

In [6]:
shelf_re ='Shelf|CC Rack|Deleted Items- Adjust|Deleted Item|Added Item'
line_re = re.compile(r'\d')
all_record = []
with pdfplumber.open(PDF_FILE) as pdf:
        pages = pdf.pages
        shelf_record = []
        page_shelf_record_dict = {}
        table_number = 0
        shelf_num_index = ''
        for i,page in enumerate(pages):
            if i>0:
                page_number = "page_"+str(i)
                text = page.extract_text()
                shelf_record_dict = {}
                
                for line in text.split('\n'):
                    shelf_number = re.findall(shelf_re, line, flags=re.IGNORECASE)
                    if shelf_number:
                        shelf_num_index = re.search(shelf_number[0]+'\s*([a-zA-Z0-9_.-]*)',line).group()
                    else:
                        if line.startswith('Loc. ID '):
                            table_number = table_number+1
                            print("##### Start a new Table#####")

                        elif line_re.search(line) and (table_number>0):
                            line_desc = get_table_info(shelf_num_index,line)
                            if line_desc:
                                all_record.append(line_desc)
                            print(line_desc)
print(all_record)

##### Start a new Table#####
['Shelf 1', '1', 'ARIZONA ARNOLD PALMER HALF & HALF STRAWB... ', '23.00', 'OZ', 'v(g13008*RNPNKm(W', '7635', '1']
['Shelf 1', '2', 'ARIZONA ARNOLD PALMER HALF & HALF TEA/LEM... ', '23.00', 'OZ', 'v(g13008*RLTNKm(W', '7616', '1']
['Shelf 1', '3', 'ARIZONA RX ENERGY ', '23.00', 'OZ', 'v(g13008*RLSOPt(y', '7614', '1']
['Shelf 1', '4', 'ARIZONA SWEET TEA CAN ', '23.00', 'OZ', 'v(g13008*RLRRLl(V', '7612', '1']
['Shelf 1', '5', 'ARIZONA RASPBERRY TEA ', '23.00', 'OZ', 'v(g13008*RNPORk(U', '7611', '1']
['Shelf 1', '6', 'ARIZONA GREEN TEA ', '23.00', 'OZ', 'v(g13008*RLPMQr(w', '7610', '1']
['Shelf 1', '7', 'ARIZONA MUCHO MANGO TALL BOY 20 OZ ', '20.00', 'OZ', 'v(g13008*RPLNNn(X', '924835', '1']
['Shelf 1', '8', 'LIPTON GREEN TEA W/CITRUS ', '20.00', 'OZ', 'U(a12000*KLMRPo(Y', 'PEPSI', '1']
['Shelf 1', '9', 'LIPTON PURE LEAF RASPBERRY TEA ', '18.50', 'OZ', 'U(a12000*MSQMMn(X', 'PEPSI', '1']
##### Start a new Table#####
['Shelf 2', '10', 'KWIKADE ARCTIC FREEZE ', '12

[]
['Shelf 15', '99', 'KC COLBY JACK CHEESE SLICES 8 OZ ', '8.00', 'OZ', 'U(a39779*KKKQNs(x', '920888', '1']
['Shelf 15', '100', 'KC STRING CHEESE REGULAR 12 OZ BAG ', '12.00', 'OZ', 'U(a39779*KMKQRo(Y', '20672', '1']
##### Start a new Table#####
['Shelf 16', '101', 'KEMPS SELECT HEAVY WHIPPED CREAM 13 OZ ', '12.00', 'CT', 'U(a41483*KOMQKq(v', '925331', '1']
['Shelf 16', '102', 'KEMPS HEAVY WHIPPING CREAM ', '8.00', 'OZ', 'U(a41483*KKQOQm(W', '111303', '1']
['Shelf 16', '103', 'INT DELIGHT FR VAN ', '16.000', 'Z', 'U(a41271*KMPQMk(U', '20636', '1']
['Shelf 16', '104', 'NT CINN HAZELNUT CREAMER ', '16.00', 'OZ', 'U(a39779*LLLMNp(u', '111206', '1']
['Shelf 16', '105', 'NT IRISH CREAM CREAMER ', '16.00', 'OZ', 'U(a39779*LLLNMr(w', '111207', '1']
['Shelf 16', '106', 'NT FRENCH VANILLA CREAMER ', '16.00', 'OZ', 'U(a39779*LLLMMs(x', '111205', '1']
['Shelf 16', '107', 'NT 1/2 & 1/2 PINT ', '16.00', 'OZ', 'U(a39779*LLLMLl(V', '111204', '1']
['Shelf 16', '108', "NATURE'S TOUCH FRENCH VANILLA CR

**Export into CSV or TSV**

In [7]:
df = pd.DataFrame(all_record,columns=['Shelf No.', 'Loc.ID', 'Name', 'Size', 'UOM', 'UPC-Barcode', 'Stockcode', 'Facings'])
csv_file = PDF_FILE.split('.')[0] + '.table.csv'
tsv_file = PDF_FILE.split('.')[0] + '.table.tsv'
df.to_csv(tsv_file,sep='\t', index=False)
df.to_csv(csv_file, index=False)